# 1. Initialization

In [1]:
import pandas as pd
from openhexa.sdk import workspace
from tqdm import tqdm
import warnings
import os
from os import listdir, environ
from copy import deepcopy
from os.path import isfile, join
from sqlalchemy import create_engine

In [2]:
tqdm.pandas()
warnings.filterwarnings("ignore")
local = True

# 2. Creating the statistics

## 2.1 Define the necessary parameters

Variable name changes:

**Folder names**
- result_simulation --> simulation_statistics
- Selections_Verif --> verification_information

**Parameters in the file names**
(These I changed because it was complaning about the file names being too long,
not because I find them clearer).
- GAIN_VERIF_MEDIAN_MAX --> gain_verif
- MIN_NB_TRIM_OBS --> obs_win
- MIN_NB_TRIM_WITH_VERIF --> min_nb_verif
- seuil_max_bas_risk --> seuil_b
- seuil_max_moyen_risk --> seuil_m
- Paiement --> pai
- Quality_risk --> qual_risk

**Column names in Selections_Verif / verification_information**
- verified --> bool_verified

**Column names in result_simulation / simulation_statistics**
- cout total (VBR) --> total cost (VBR)
- cout total (syst) --> total cost (syst)

In [3]:
if local:
    data_path = os.path.join(os.getcwd(), "data")
else:
    data_path = f"{workspace.files_path}/PBF burundi extraction/data"

DB_names = {
    os.path.join(data_path, "verification_information"): "VBR_liste_detaillees",
    os.path.join(data_path, "simulation_statistics"): "VBR_results",
}

dict_params = {
    "gain_verif": [],
    "obs_win": [],
    "min_nb_verif": [],
    "p_low": [],
    "p_mod": [],
    "p_high": [],
    "cout_verif": [],
    "seuil_b": [],
    "seuil_m": [],
    "pai": [],
    "qual_risk": [],
}

## 2.2 Define the necessary functions

In [4]:
def valid_simulation_name(filename):
    return all(substring in filename for substring, _ in dict_params.items())


def get_parameters(f):
    dict_params_full = deepcopy(dict_params)
    dict_params_full["name"] = f
    dict_params_full["model"] = ["defaut"]
    list_file_params = f.split("-")
    for p in dict_params:
        for p_match in list_file_params:
            if p in p_match:
                dict_params_full.setdefault(p, []).append(
                    p_match.split("___")[1].replace(".csv", "")
                )
    return pd.DataFrame.from_dict(dict_params_full)


def get_statistics(mypath, f):
    df = pd.read_csv(join(mypath, f))
    df["name"] = f
    return df


def process_verification_info(df):
    df["nb_centers_verified"] = df["bool_verified"].map(lambda x: 1 if x else 0)
    df["nb_centers"] = 1

    df["#_scores_risque_eleve"] = df["categorie_risque"].map(
        lambda x: 1 if x == "high" or x == "uneligible" else 0
    )
    df["#_scores_risque_mod1"] = df["categorie_risque"].map(lambda x: 1 if x == "moderate_1" else 0)
    df["#_scores_risque_mod2"] = df["categorie_risque"].map(lambda x: 1 if x == "moderate_2" else 0)
    df["#_scores_risque_mod3"] = df["categorie_risque"].map(lambda x: 1 if x == "moderate_3" else 0)
    df["#_scores_risque_mod"] = df["categorie_risque"].map(lambda x: 1 if x == "moderate" else 0)
    df["#_scores_risque_faible"] = df["categorie_risque"].map(lambda x: 1 if x == "low" else 0)
    # These moderate_1, moderate_2, etc should probably be in a config file instead of hard coded here...
    df = df.groupby(
        [
            "period",
            "model",
            "p_high",
            "p_mod",
            "p_low",
            "min_nb_verif",
            "obs_win",
            "gain_verif",
            "pai",
            "name",
            "level_2_name",
            "level_3_name",
        ],
        as_index=False,
    )[
        [
            "nb_centers",
            "nb_centers_verified",
            "#_scores_risque_faible",
            "#_scores_risque_mod1",
            "#_scores_risque_mod2",
            "#_scores_risque_mod3",
            "#_scores_risque_eleve",
            "#_scores_risque_mod",
        ]
    ].sum()
    return df


def str_to_date(datestr):
    if isinstance(datestr, str) and "Q" in datestr:
        return datestr[:4] + "-" + str(int(datestr[5]) * 3) + "-1"
    elif isinstance(datestr, int) or datestr.isdigit():
        x = int(datestr)
        return f"{x // 100}-{x % 100}-1"

## 2.3 Create the output csvs

In [5]:
for mypath, _ in DB_names.items():
    # initialize the variables
    dfs = pd.DataFrame()
    dfs_detailled = pd.DataFrame()
    list_files = [
        f for f in listdir(mypath) if isfile(join(mypath, f)) and valid_simulation_name(f)
    ]

    # Get the files in the path
    for filename in list_files:
        df = get_parameters(filename).merge(get_statistics(mypath, filename), on="name")

        if "verification_information" in mypath:
            dfs_detailled = pd.concat([dfs_detailled, df], ignore_index=True)
            df = process_verification_info(df)
            
        dfs = pd.concat([dfs, df], ignore_index=True)

    # Clean and save the csv that concatenates the information in the input files. 
    dfs.rename(columns={"level_2_name": "province", "period": "periode"}, inplace=True)
    dfs = dfs.sort_values(["province", "periode"]).drop("name", axis=1)
    if "simulation_statistics" in mypath:
        dfs["gain_vbr"] = dfs["total cost (syst)"] - dfs["total cost (VBR)"]
    if not local:
        engine = create_engine(environ["WORKSPACE_DATABASE_URL"])
        dfs.to_sql(DB_names[mypath], con=engine, if_exists="replace")
    dfs.to_csv(f"{data_path}/{DB_names[mypath]}.csv", index=False)
    display(f"For the {DB_names[mypath]}, the columns are:")
    display(dfs.columns.values)

    # For the verification_information file, we also save the information in a detailed way. 
    if "verification_information" in mypath:
        dfs_detailled["bool_verified"] = dfs_detailled["bool_verified"].astype(int)
        dfs_detailled.rename(
            columns={"level_2_name": "province", "period": "periode"}, inplace=True
        )
        dfs_detailled["date"] = dfs_detailled["periode"].map(str_to_date)
        dfs_detailled = dfs_detailled.sort_values(["province", "periode"]).drop("name", axis=1)
        dfs_detailled.to_csv(f"{data_path}/VBR_liste_verification.csv", index=False)
        if not local:
            dfs_detailled.to_sql("VBR_liste_verification", con=engine, if_exists="replace")
        display("For the VBR_liste_verification, the columns are:")
        display(dfs_detailled.columns.values)


'For the VBR_liste_detaillees, the columns are:'

array(['periode', 'model', 'p_high', 'p_mod', 'p_low', 'min_nb_verif',
       'obs_win', 'gain_verif', 'pai', 'province', 'level_3_name',
       'nb_centers', 'nb_centers_verified', '#_scores_risque_faible',
       '#_scores_risque_mod1', '#_scores_risque_mod2',
       '#_scores_risque_mod3', '#_scores_risque_eleve',
       '#_scores_risque_mod'], dtype=object)

'For the VBR_liste_verification, the columns are:'

array(['gain_verif', 'obs_win', 'min_nb_verif', 'p_low', 'p_mod',
       'p_high', 'cout_verif', 'seuil_b', 'seuil_m', 'pai', 'qual_risk',
       'model', 'periode', 'ou_id', 'level_2_uid', 'province',
       'level_3_uid', 'level_3_name', 'level_4_uid', 'level_4_name',
       'level_5_uid', 'level_5_name', 'level_6_uid', 'level_6_name',
       'bool_verified', 'diff_in_subsidies_decval_period',
       'diff_in_subsidies_tauxval_period', 'benefice_complet_vbr',
       'taux_validation', 'subside_dec_period', 'subside_val_period',
       'subside_taux_period', 'ecart_median', 'categorie_risque', 'date'],
      dtype=object)

'For the VBR_results, the columns are:'

array(['gain_verif', 'obs_win', 'min_nb_verif', 'p_low', 'p_mod',
       'p_high', 'cout_verif', 'seuil_b', 'seuil_m', 'pai', 'qual_risk',
       'model', 'province', 'periode', 'total number of centres',
       'number of centers high risk', 'number of centers middle risk',
       'number of centers low risk', 'number of verified centers',
       'cost of verification (VBR)', 'cost of verification (syst)',
       'subsidies (VBR)', 'subsidies (syst)', 'total cost (VBR)',
       'total cost (syst)', 'ratio cost_verification cost_total (VBR)',
       'ratio cost_verification cost_total (syst)',
       'Number of centers where vbr was beneficial',
       'Number of centers over-subsidized',
       'Money saved by VBR (taking into account verif costs)',
       'Amount of over-subsidies',
       'Average of over-subsidies (non-verified centers)',
       'Average of over-subsidies (verified centers)', 'gain_vbr'],
      dtype=object)